In [ ]:
prompt_resume = f"""
  <|im_start|>system
  Описание роли ЛЛМ
  и задачи, которую мы ставим
  one-shot и few-shot сюда же
  <|im_end|>

  <|im_start|>user
  Вопрос к ЛЛМ
  <|im_end|>
  <|im_start|>assistant
  """
# все служебные команды модель красиво форматирует

Загрузка данных

In [ ]:
from datasets import load_dataset

ds = load_dataset("JeswinMS4/code_text_classification")

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'Label', '__index_level_0__'],
        num_rows: 823
    })
    test: Dataset({
        features: ['sentence', 'Label', '__index_level_0__'],
        num_rows: 46
    })
    validation: Dataset({
        features: ['sentence', 'Label', '__index_level_0__'],
        num_rows: 46
    })
})

Загрузка модели

In [ ]:
  pip install bitsandbytes -q

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

print("Model successfully loaded on device:", model.device)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else cpu)
model.to(device);

Подготовка данных

In [ ]:
def data_preparation(train_size, test_size):
  sample_train = ds["train"].select(range(train_size))
  sample_df_train = sample_train.to_pandas()
  sample_df_train['bin'] = sample_df_train['Label'].apply(lambda x: 1 if x=='Code Generation' else 0)

  sample_test = ds["test"].select(range(test_size))
  sample_df_test = sample_test.to_pandas()
  sample_df_test['bin'] = sample_df_test['Label'].apply(lambda x: 1 if x=='Code Generation' else 0)
  return sample_df_train, sample_df_test
sample_df_train, sample_df_test = data_preparation(100, 46)
print(sample_df_train.shape, sample_df_test.shape)

(100, 4) (46, 4)


In [ ]:
len(sample_df_test.sentence.unique())

46

In [ ]:
# FewShotInstructions
print(sample_df_train.shape)
few_shot_instructions = []
drop_ids = []
for lbl in range(2):
  for num in range(2):
    few_shot_instruction = ' '.join([
                                      "<|im_start|>user\n ",
                                      "Это запрос на генерацию кода (1) или текста (0)? Ответь ТОЛЬКО 1 или 0.\n",
                                      f"{sample_df_train[sample_df_train.bin==lbl]['sentence'].iloc[num]} \n",
                                      "<|im_end|>\n",
                                      "<|im_start|>assistant \n",
                                      str(lbl),
                                      "\n",
                                      ])
    drop_ids.append(sample_df_train[sample_df_train.bin==lbl].iloc[num]['__index_level_0__'])
    few_shot_instructions.append(few_shot_instruction)
few_shot_instructions = '\n'.join(few_shot_instructions)
sample_df_train = sample_df_train[~sample_df_train['__index_level_0__'].isin(drop_ids)]
sample_df_train.shape

(100, 4)


(96, 4)

Пример вывода модели

In [ ]:
text = sample_df_train['sentence'].iloc[0]
prompt = f"""

  <|im_start|>system
  You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
  Ты — классификатор текстов.
  Задача: Классификация запросов: запрос на генерацию кода (1), либо генерацию текста (0).
  Отвечай ТОЛЬКО "1" или "0", без объяснений.<|im_end|>

  <|im_start|>user
  Это запрос на генерацию кода (1) или текста (0)? Ответь ТОЛЬКО 1 или 0.
  {text}
  <|im_end|>
  <|im_start|>assistant
  """
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.9)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)



  system
  You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
  Ты — классификатор текстов.
  Задача: Классификация запросов: запрос на генерацию кода (1), либо генерацию текста (0).
  Отвечай ТОЛЬКО "1" или "0", без объяснений.

  user
  Это запрос на генерацию кода (1) или текста (0)? Ответь ТОЛЬКО 1 или 0.
  What is the chemical formula for carbon monoxide?
  
  assistant
  0


Строим модель

In [ ]:
from tqdm import tqdm

In [ ]:
# Inner Prompt Test  {few_shot_instructions}
y_pred = []
y_true = []
for cnt in (range(sample_df_train.shape[0])):
  text = sample_df_train['sentence'].iloc[cnt]
  prompt = f"""

  <|im_start|>system
  You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
  Ты — классификатор текстов.
  Задача: Классификация запросов: запрос на генерацию кода (1), либо генерацию текста (0).
  Отвечай ТОЛЬКО "1" или "0", без объяснений.<|im_end|>

  <|im_start|>user
  Это запрос на генерацию кода (1) или текста (0)? Ответь ТОЛЬКО 1 или 0.
  {text}
  <|im_end|>
  <|im_start|>assistant
  """
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.9)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # print(result)
  if "0" not in result and "1" not in result:
    print("Модель ответила неожиданно.")
    result = None

  # print(result)
  # print('raw', result.split('<im_start>assistant\n')[-1].split())
  # print('cut', result.split('<im_start>assistant\n')[-1].split()[0])

  try:
    # print('pred:', int(resu split('<im_start>assistant\n')[-1].split()[0]), 'true:', sample_df_test['bin'].iloc[cnt])
    y_pred.append(int(result.split('assistant\n')[-1].split()[0]))
    y_true.append(int(sample_df_train['bin'].iloc[cnt]))
  except:
    # print(result.split('assistant\n')[-1].split()[0])
    print('\n \n smth wrong', cnt)
  # print('---'*20)

sum_d2 = []
for i in range(len(y_pred)):
  sum_d2.append((y_pred[i]+y_true[i])/2)
from collections import Counter
cntr = Counter(sum_d2)
Accuracy = (cntr[0.0]+cntr[1.0])/len(sum_d2)
Accuracy

0.5625

In [ ]:
# Inner Prompt Test  {few_shot_instructions}
y_pred = []
y_true = []
for cnt in (range(sample_df_train.shape[0])):
  text = sample_df_train['sentence'].iloc[cnt]
  prompt = f"""
  <|im_start|>system
  Ты — классификатор текстов.
  Задача: Классификация запросов: запрос на генерацию кода (1), либо генерацию текста (0).
  Отвечай ТОЛЬКО "1" или "0", без объяснений.<|im_end|>

  <|im_start|>user
  Это запрос на генерацию кода (1) или текста (0)? Ответь ТОЛЬКО 1 или 0.
  {text}
  <|im_end|>
  <|im_start|>assistant
  """
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.9)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # print(result)
  if "0" not in result and "1" not in result:
    print("Модель ответила неожиданно.")
    result = None

  # print(result)
  # print('raw', result.split('<im_start>assistant\n')[-1].split())
  # print('cut', result.split('<im_start>assistant\n')[-1].split()[0])

  try:
    # print('pred:', int(resu split('<im_start>assistant\n')[-1].split()[0]), 'true:', sample_df_test['bin'].iloc[cnt])
    y_pred.append(int(result.split('assistant\n')[-1].split()[0]))
    y_true.append(int(sample_df_train['bin'].iloc[cnt]))
    print(len(y_pred), len(y_true), end='---')
  except:
    print('smth wrong', cnt)
  # print('---'*20)

sum_d2 = []
for i in range(len(y_pred)):
  sum_d2.append((y_pred[i]+y_true[i])/2)
from collections import Counter
cntr = Counter(sum_d2)
Accuracy = (cntr[0.0]+cntr[1.0])/len(sum_d2)
Accuracy

1 1---2 2---3 3---4 4---5 5---6 6---7 7---8 8---9 9---10 10---smth wrong 10
11 11---12 12---13 13---14 14---15 15---16 16---17 17---18 18---19 19---20 20---21 21---22 22---23 23---24 24---25 25---26 26---27 27---28 28---29 29---30 30---31 31---32 32---33 33---34 34---35 35---36 36---37 37---38 38---39 39---40 40---41 41---42 42---43 43---44 44---45 45---46 46---47 47---48 48---49 49---50 50---51 51---52 52---53 53---54 54---55 55---56 56---57 57---58 58---59 59---60 60---61 61---62 62---63 63---64 64---65 65---66 66---67 67---68 68---69 69---70 70---71 71---72 72---73 73---74 74---75 75---76 76---77 77---78 78---79 79---80 80---81 81---82 82---83 83---84 84---85 85---86 86---87 87---88 88---89 89---90 90---91 91---92 92---93 93---94 94---95 95---

0.6526315789473685

In [ ]:
# Inner Prompt Test  {few_shot_instructions}
y_pred = []
y_true = []
for cnt in (range(sample_df_train.shape[0])):
  text = sample_df_train['sentence'].iloc[cnt]
  prompt = f"""
  <|im_start|>system
  Ты — классификатор текстов.
  Задача: Классификация запросов: запрос на генерацию кода (1), либо генерацию текста (0).
  Отвечай ТОЛЬКО "1" или "0", без объяснений.

  Примеры правильных ответов:
  {few_shot_instructions}
  Конец примеров.
  <|im_end|>

  <|im_start|>user
  Это запрос на генерацию кода (1) или текста (0)? Ответь ТОЛЬКО 1 или 0.
  {text}
  <|im_end|>
  <|im_start|>assistant
  """
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.9)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # print(result)
  if "0" not in result and "1" not in result:
    print("Модель ответила неожиданно.")
    result = None

  # print(result)
  # print('raw', result.split('<im_start>assistant\n')[-1].split())
  # print('cut', result.split('<im_start>assistant\n')[-1].split()[0])

  try:
    # print('pred:', int(resu split('<im_start>assistant\n')[-1].split()[0]), 'true:', sample_df_test['bin'].iloc[cnt])
    y_pred.append(int(result.split('assistant\n')[-1].split()[0]))
    y_true.append(int(sample_df_train['bin'].iloc[cnt]))
    print(len(y_pred), len(y_true), end='---')
  except:
    print('smth wrong', cnt)
  # print('---'*20)

sum_d2 = []
for i in range(len(y_pred)):
  sum_d2.append((y_pred[i]+y_true[i])/2)
from collections import Counter
cntr = Counter(sum_d2)
Accuracy = (cntr[0.0]+cntr[1.0])/len(sum_d2)
Accuracy

1 1---2 2---3 3---4 4---5 5---6 6---7 7---8 8---9 9---10 10---11 11---12 12---13 13---14 14---15 15---16 16---17 17---18 18---19 19---20 20---21 21---22 22---23 23---24 24---25 25---26 26---27 27---28 28---29 29---30 30---31 31---32 32---33 33---34 34---35 35---36 36---37 37---38 38---39 39---40 40---41 41---42 42---43 43---44 44---45 45---46 46---47 47---48 48---49 49---50 50---51 51---52 52---53 53---54 54---55 55---56 56---57 57---58 58---59 59---60 60---61 61---62 62---63 63---64 64---65 65---66 66---67 67---68 68---69 69---70 70---71 71---72 72---73 73---74 74---75 75---76 76---77 77---78 78---79 79---80 80---81 81---82 82---83 83---84 84---85 85---86 86---87 87---88 88---89 89---90 90---91 91---92 92---93 93---94 94---95 95---96 96---

0.84375

In [ ]:
# Inner Prompt Test  {few_shot_instructions}
y_pred = []
y_true = []
for cnt in (range(sample_df_test.shape[0])):
  text = sample_df_test['sentence'].iloc[cnt]
  prompt = f"""
  <|im_start|>system
  Ты — классификатор текстов.
  Задача: Классификация запросов: запрос на генерацию кода (1), либо генерацию текста (0).
  Отвечай ТОЛЬКО "1" или "0", без объяснений.

  Примеры правильных ответов:
  {few_shot_instructions}
  Конец примеров.
  <|im_end|>

  <|im_start|>user
  Это запрос на генерацию кода (1) или текста (0)? Ответь ТОЛЬКО 1 или 0.
  {text}
  <|im_end|>
  <|im_start|>assistant
  """
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.9)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # print(result)
  if "0" not in result and "1" not in result:
    print("Модель ответила неожиданно.")
    result = None

  # print(result)
  # print('raw', result.split('<im_start>assistant\n')[-1].split())
  # print('cut', result.split('<im_start>assistant\n')[-1].split()[0])

  try:
    # print('pred:', int(resu split('<im_start>assistant\n')[-1].split()[0]), 'true:', sample_df_test['bin'].iloc[cnt])
    y_pred.append(int(result.split('assistant\n')[-1].split()[0]))
    y_true.append(int(sample_df_test['bin'].iloc[cnt]))
    print(len(y_pred), len(y_true), end='---')
  except:
    print('smth wrong', cnt)
  # print('---'*20)

sum_d2 = []
for i in range(len(y_pred)):
  sum_d2.append((y_pred[i]+y_true[i])/2)
from collections import Counter
cntr = Counter(sum_d2)
Accuracy = (cntr[0.0]+cntr[1.0])/len(sum_d2)
Accuracy

1 1---2 2---3 3---4 4---5 5---6 6---7 7---8 8---9 9---10 10---11 11---12 12---13 13---14 14---15 15---16 16---17 17---18 18---19 19---20 20---21 21---22 22---23 23---24 24---25 25---26 26---27 27---28 28---29 29---30 30---31 31---32 32---33 33---34 34---35 35---36 36---37 37---38 38---39 39---40 40---41 41---42 42---43 43---44 44---45 45---46 46---

0.8478260869565217